In [1]:
from os.path import abspath
from pyspark.sql import SparkSession,DataFrame
from datetime import datetime


# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [12]:
warehouse_location

'/home/jovyan/work/sql/spark-warehouse'

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d0182cb-b942-44c7-97ae-026a04db01ac;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [4]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser

In [5]:
######## modulo utils inicio ########
# from pyspark.sql import SparkSession, DataFrame
# import pyspark.sql.functions as f
# from dateutil import parser
# from delta.tables import *
# from loguru import logger
# from datetime import datetime

def generete_symlink_manifest(spark:SparkSession, bucket:str) -> None:
    """
    Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
    """
    delta_table = DeltaTable.forPath(spark, bucket)
    delta_table.generate("symlink_format_manifest")

def path_exists(path, spark:SparkSession):
    """
    Dado um caminho verifica se existe ou não
    """
    sc = spark.sparkContext
    fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
        sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
        sc._jsc.hadoopConfiguration(),
    )
    return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))

def last_date(delta_path:str, spark:SparkSession):
    """
    Retorna a data (timestamp_kafka) do último registro da tabela Delta
    """
    if not path_exists(delta_path + 'job_control/last_date', spark):
        try:
            df_temp = (
                spark
                .read
                .format("delta")
                .load(delta_path)
            )

            list_result = (
                df_temp
                .agg({"timestamp_kafka": "max"}).collect()
            )

            last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
            snapshot_flag = False
        except:
            snapshot_flag = True
            last_date = ('01-01-2021 12:00AM')

        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
    else:
        snapshot_flag = False
        df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
        last_date = df.collect()[0].asDict()['last_date']

    return parser.parse(last_date),snapshot_flag

def update_last_date(delta_path:str, df:DataFrame, spark:SparkSession):
    """
    Atualiza a data (timestamp_kafka) do último registro da tabela Delta
    """
    list_result = (
        df
        .agg({"timestamp_kafka": "max"}).collect()
    )

    last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
    df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
    df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')

    return last_date

def get_operations_counts(df:DataFrame, spark:SparkSession):
    """
    Retorna o número de registros de cada operação kafka (__op)
    """
    op_count_list =(
        df
        .groupBy('__op')
        .count()
        .collect()
    )

    new_data_count = 0
    updating_data_count = 0
    deleting_data_count = 0

    if len(op_count_list) > 0:
        for i in op_count_list:
            dic_temp = i.asDict()
            if (dic_temp['__op'] == 'r'):
                new_data_count = new_data_count + dic_temp['count']
            elif (dic_temp['__op'] != 'r'):
                updating_data_count = updating_data_count + dic_temp['count']
            elif (dic_temp['__op'] == 'd'):
                deleting_data_count = deleting_data_count + dic_temp['count']

    new_total_data_count = new_data_count + updating_data_count + deleting_data_count

    return new_total_data_count, new_data_count, updating_data_count, deleting_data_count

def get_data(spark:SparkSession, bucket:str, date_colunm_name:str, date_begin:datetime, source_format:str='delta'):
    """
    Carrega Dataframe filtrando os dados do bucket de referência conforme 
    data de início enviada no parametro

    :param str date_begin: Data de início para filtro dos dados
    """
    df = (
        spark
        .read
        .format(source_format)
        .load(bucket)
        .where(
            (f.col("year")>date_begin.year)|
            (
                (f.col("year")==date_begin.year)&
                (
                (f.col("month")>date_begin.month)|
                (
                    (f.col("month")==date_begin.month)&
                    (
                    (f.col("day")>date_begin.day)|
                    (
                        (f.col("day")==date_begin.day)&
                        (f.col("hour")>=date_begin.hour)&
                        (f.col(date_colunm_name)>date_begin)
                    )
                    )
                )
                )          
            ) 
        )
    )
    return df

def write_new_data(spark:SparkSession, df:DataFrame, bucket:str, destination_format:str='delta'):
    """
    Escreve no bucket de destino os dados novos, cuja a operação kafka é 'r'
    """
    df_r = df.where((f.col('__op') == 'r'))
    (
        df_r
        .write
        .format(destination_format)
        .mode("append")
        .option("checkpointLocation", bucket + "checkpoint")
        .save(bucket)
    )

    return True

def update_new_data(spark:SparkSession, df:DataFrame, bucket:str, pk:list=['pk_single']):
    """
    Escreve no bucket de destino os dados novos, cuja a operação kafka é 'c' e
    atualiza os dados existentes, cuja a operação kafka é 'u' ou 'd'
    """
    table = DeltaTable.forPath(spark, bucket)
    df_table_u = df.where((f.col('__op') != 'r'))
    df_table_u_unique = df_table_u.orderBy('timestamp_kafka', ascending = False).dropDuplicates(pk)
    merge_string = 'destination.' + pk[0] + ' = updates.' + pk[0]
    (
        table.alias('destination')
        .merge(df_table_u_unique.alias('updates'), merge_string)
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )

    return True

def delete_new_data(spark:SparkSession, bucket:str):
    """
    Deleta da tabela delta, no bucket de destino, os dados cuja a operação kafka é 'd'
    """
    table = DeltaTable.forPath(spark, bucket)
    table.delete(f.col('__op') == 'd')

    return True

def flatten_df(nested_df):
    """
    Retorna um dataframe com as colunas de um dataframe aninhado
    """
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

    flat_df = nested_df.select(flat_cols +
                            [f.col(nc+'.'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df

def write_bronze_table(df:DataFrame, bucket:str):
    """
    Escreve no bucket de destino a tabela delta bronze
    """
    (
        df
        .write
        .partitionBy("year", "month", "day", "hour")
        .format("delta")
        .mode("append")
        .option("checkpointLocation", bucket + "checkpoint")
        .save(bucket)
    )

    return True
########  modulo utils fim   ########

In [6]:
topic = 'bnsvher602.linx_db.dbo.HR_OMNI_PEDIDO'
landing_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido/'
bronze_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido/'
url = 'http://schema-registry-cp-schema-registry.ingestion.svc:8081/subjects/' + topic + '-value/versions/latest'

In [7]:
response = urlopen(url)
data_json = json.loads(response.read())

In [8]:
last_date, snapshot_flag = last_date(bronze_path, spark)

In [9]:
snapshot_flag

True

In [10]:
last_date

datetime.datetime(2021, 1, 1, 0, 0)

In [11]:
df_table = get_data(spark, landing_path, 'timestamp', last_date, 'avro')
df_table = df_table.withColumnRenamed("timestamp", "timestamp_kafka")

In [12]:
df_table.count()

1139068

In [13]:
from_avro_options= {"mode":"PERMISSIVE"}
df_temp = (
    df_table
    .withColumn('fixedValue', f.expr("substring(value, 6, length(value)-5)"))
    .select(from_avro("fixedValue", data_json['schema'], from_avro_options), 'offset', 'timestamp_kafka',"year", "month", "day", "hour")
)


In [14]:
df_table_flat = flatten_df(df_temp)

In [16]:
df_table_flat.limit(5).toPandas()

offset         timestamp_kafka  year  month  day  hour CODIGO_SITE  \
0   91586 2022-03-15 13:37:18.696  2022      3   15    13         001   
1   91587 2022-03-15 13:37:18.696  2022      3   15    13         001   
2   91588 2022-03-15 13:37:18.697  2022      3   15    13         006   
3   91589 2022-03-15 13:37:18.697  2022      3   15    13         006   
4   91590 2022-03-15 13:37:18.697  2022      3   15    13         006   

             PEDIDO           DATA  TIPO_PEDIDO  ... DATA_PARA_TRANSFERENCIA  \
0         210311018  1521072000000            1  ...           1552046596747   
1         299501668  1543017600000            1  ...           1552046596747   
2  1217001017765-01  1647043200000            4  ...           1647105520677   
3  1217001017765-01  1647043200000            4  ...           1647105520677   
4  1217001017765-01  1647043200000            4  ...           1647105520677   

  EXISTE_PRECO_BANDA_ALTA  INTEGRACAO_CANCELADA  \
0                    None                  None   
1                    None                  None   
2                    None                  None   
3                   False                  None   
4                   False                  None   

   DATA_HORA_INTEGRACAO_CANCELADA  DEVE_FATURAR  PEDIDO_DIFERENTE_RESERVA  \
0                             NaN          None                      True   
1                             NaN          None                      True   
2                             NaN          None                      True   
3                             NaN          None                      True   
4                             NaN          None                      True   

  __op         __table  __source_ts_ms __deleted  
0    u  HR_OMNI_PEDIDO   1647125986860     false  
1    u  HR_OMNI_PEDIDO   1647125986860     false  
2    u  HR_OMNI_PEDIDO   1647125986860     false  
3    u  HR_OMNI_PEDIDO   1647125986860     false  
4    u  HR_OMNI_PEDIDO   1647125986860     false  

[5 rows x 39 columns]

In [ ]:
df_table.count()

In [17]:
print('Defining variables for snapshot')
landing_path_snapshot = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido_snapshot/'
topic_snapshot = 'schema.pedido.bnsvher602.linx_db.dbo.HR_OMNI_PEDIDO'
url_snapshot = 'http://schema-registry-cp-schema-registry.ingestion.svc:8081/subjects/' + topic_snapshot + '-value/versions/latest'

Defining variables for snapshot


In [18]:
print('Getting snapshot schema from schema registry')
response_snapshot = urlopen(url_snapshot)
data_json_snapshot = json.loads(response_snapshot.read())

Getting snapshot schema from schema registry


Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:458)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [19]:
data_json_snapshot['schema']

'{"type":"record","name":"Value","namespace":"schema.pedido.bnsvher602.linx_db.dbo.HR_OMNI_PEDIDO","fields":[{"name":"CODIGO_SITE","type":["null","string"],"default":null},{"name":"PEDIDO","type":"string"},{"name":"DATA","type":["null",{"type":"long","connect.version":1,"connect.name":"io.debezium.time.Timestamp"}],"default":null},{"name":"TIPO_PEDIDO","type":["null","int"],"default":null},{"name":"CODIGO_CLIENTE","type":["null","string"],"default":null},{"name":"ID_ENDERECO_COBRANCA","type":["null","string"],"default":null},{"name":"QTDE_TOTAL","type":["null","int"],"default":null},{"name":"VALOR_TOTAL","type":["null","double"],"default":null},{"name":"DESCONTO","type":["null","double"],"default":null},{"name":"VALOR_FRETE","type":["null","double"],"default":null},{"name":"CODIGO_DESCONTO","type":["null","string"],"default":null},{"name":"VENDEDOR_PRINCIPAL","type":["null","string"],"default":null},{"name":"DATA_HORA_PEDIDO","type":["null",{"type":"long","connect.version":1,"connect.n

In [20]:
df_table_snapshot = get_data(spark, landing_path_snapshot, 'timestamp', last_date, 'avro')
df_table_snapshot = df_table_snapshot.withColumnRenamed("timestamp", "timestamp_kafka")


In [ ]:
print(df_table_snapshot.count())

In [21]:
df_table_snapshot.limit(5).toPandas()

key  \
0  [0, 0, 0, 0, 15, 20, 49, 48, 48, 46, 51, 50, 7...   
1  [0, 0, 0, 0, 15, 20, 77, 76, 48, 48, 57, 54, 5...   
2  [0, 0, 0, 0, 15, 22, 49, 48, 48, 48, 46, 50, 5...   
3  [0, 0, 0, 0, 15, 20, 77, 76, 48, 48, 57, 54, 5...   
4  [0, 0, 0, 0, 15, 22, 49, 48, 48, 48, 46, 50, 6...   

                                               value  \
0  [0, 0, 0, 0, 16, 2, 6, 48, 48, 49, 20, 49, 48,...   
1  [0, 0, 0, 0, 16, 2, 6, 48, 48, 49, 20, 77, 76,...   
2  [0, 0, 0, 0, 16, 2, 6, 48, 48, 49, 22, 49, 48,...   
3  [0, 0, 0, 0, 16, 2, 6, 48, 48, 49, 20, 77, 76,...   
4  [0, 0, 0, 0, 16, 2, 6, 48, 48, 49, 22, 49, 48,...   

                                        topic  partition   offset  \
0  temp_bnsvher602.linx_db.dbo.HR_OMNI_PEDIDO          0        0   
1  temp_bnsvher602.linx_db.dbo.HR_OMNI_PEDIDO          0 -2177277   
2  temp_bnsvher602.linx_db.dbo.HR_OMNI_PEDIDO          0       -1   
3  temp_bnsvher602.linx_db.dbo.HR_OMNI_PEDIDO          0 -2177278   
4  temp_bnsvher602.linx_db.dbo.HR_OMNI_PEDIDO          0       -2   

          timestamp_kafka  timestampType  year  month  day  hour  
0 2022-03-10 17:28:55.796              0  2022      3   10    17  
1 2022-03-10 17:43:41.909              0  2022      3   10    17  
2 2022-03-10 17:28:55.803              0  2022      3   10    17  
3 2022-03-10 17:43:41.909              0  2022      3   10    17  
4 2022-03-10 17:28:55.804              0  2022      3   10    17

In [22]:
print('Converting avro snapshot to flat table')
from_avro_options= {"mode":"PERMISSIVE"}
df_temp_snapshot = (
    df_table_snapshot
    .withColumn('fixedValue', f.expr("substring(value, 6, length(value)-5)"))
    .select(from_avro("fixedValue", data_json_snapshot['schema']), 'offset', 'timestamp_kafka',"year", "month", "day", "hour")
)

Converting avro snapshot to flat table


In [23]:
df_temp_snapshot.limit(5).toPandas()

from_avro(fixedValue)   offset  \
0  (001, 100.32IAJH, 1581984000000, 2, 8670617919...        0   
1  (001, ML00964084, 1621123200000, 3, 432685, CO... -2177277   
2  (001, 1000.29UAKL, 1569801600000, 2, 451283908...       -1   
3  (001, ML00964092, 1621123200000, 3, 854669, CO... -2177278   
4  (001, 1000.2B9ASN, 1573257600000, 2, 280909388...       -2   

          timestamp_kafka  year  month  day  hour  
0 2022-03-10 17:28:55.796  2022      3   10    17  
1 2022-03-10 17:43:41.909  2022      3   10    17  
2 2022-03-10 17:28:55.803  2022      3   10    17  
3 2022-03-10 17:43:41.909  2022      3   10    17  
4 2022-03-10 17:28:55.804  2022      3   10    17

In [ ]:
print('Converting avro snapshot to flat table')
from_avro_options= {"mode":"PERMISSIVE"}
df_temp_snapshot = (
    df_table_snapshot
    .withColumn('fixedValue', f.expr("substring(value, 6, length(value)-5)"))
    .select(from_avro("fixedValue", data_json_snapshot['schema'], from_avro_options), 'offset', 'timestamp_kafka',"year", "month", "day", "hour")
)

In [ ]:
df_table_flat_snapshot = flatten_df(df_temp_snapshot)

In [ ]:
df_table_flat_snapshot.limit(5).toPandas()

In [ ]:
if snapshot_flag:



        print('Getting snapshot schema from schema registry')
        response_snapshot = urlopen(url_snapshot)
        data_json_snapshot = json.loads(response_snapshot.read())

        df_table_snapshot = get_data(spark, landing_path_snapshot, 'timestamp', last_date, 'avro')
        df_table_snapshot = df_table_snapshot.withColumnRenamed("timestamp", "timestamp_kafka")
        print(df_table_snapshot.count())

        print('Converting avro snapshot to flat table')
        from_avro_options= {"mode":"PERMISSIVE"}
        df_temp_snapshot = (
            df_table_snapshot
            .withColumn('fixedValue', f.expr("substring(value, 6, length(value)-5)"))
            .select(from_avro("fixedValue", data_json_snapshot['schema'], from_avro_options), 'offset', 'timestamp_kafka',"year", "month", "day", "hour")
        )
        
        print(df_table_snapshot.count())
        
        df_table_flat_snapshot = flatten_df(df_temp_snapshot)
        
        df_table_flat = df_table_flat.union(df_table_flat_snapshot)


In [ ]:
df_table_flat.count()

In [ ]:
df_table_flat.limit(100).toPandas()

In [ ]:
df_table_flat_snapshot.limit(10).toPandas()

In [ ]:
##################################
### Verificar está funcionando ###
##################################

In [ ]:
count_bronze = (
    spark
    .read
    .format("delta")
    .load(bronze_path)
    .count()
)

print (count_bronze)

In [ ]:
landing_path_snapshot = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/ecommerce/bnsvher602/linx_db/dbo/produtos_snapshot/'

count_landing = (
    spark
    .read
    .format("avro")
    .load(landing_path)
    .count()
)

count_landing_snap = (
    spark
    .read
    .format("avro")
    .load(landing_path_snapshot)
    .count()
)


print (count_landing + count_landing_snap)